# <span style="color:darkred">Multiplicative Schwarz  using multiple strategies</span>.

<img align="left" src="Schwarz.jpg" width=120 height=120 />

### [Simon Tavener](https://www.math.colostate.edu/~tavener/) and Andre Leautaud

#### Full text for this notebook can be found at the following [link](https://www.math.colostate.edu/~tavener/andre/DD_multiphysics.pdf)

This notebook illustrates three ways of solving a system using Multiplicative Schwarz. It first creates partitions and necessary matrices, then steps through the forms of iteration. It demonstrates C Iteration (each subdomain independently), D Iteration (subdomains calculated in parallel) and  U Iteration (computes all iterations as one matrix solve). In each, we keep track of total error, iteration error, and discretization error (true and estimates). If the estimates are good, and the system was solved, the effectivty ratio will be 1. It can read all parameters and the linear system from a csv. It can also create a csv. This csv is compatible with Multiplicative Schwarz and the MatLab Live files.

#### Set path and parameters

In [147]:
# load necessary packages
import numpy as np
import math
%run DD_Utilities.ipynb
%run MS_Utilities.ipynb
iprint= 4 # amount of output desired

In [148]:
iparam= 0

In [149]:
if iparam == 0:
    n= 10  # size of system
    p= 4   # number of subomains
    b= [1,2,4,7]  # indices of starts of subdomains (1 indexed), should be in ascending order.
    e= [3,4,8,10] # indices of ends of subdomains (1 indexed), need not be in ascending order. 
                  # Setting to an integer will make each subdomain overlap the next by that number.
    K= 3  # number of iterations
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [150]:
# This example shows how to create domains that each overlap by 1 column
#small system
if iparam == 1:
    n= 8
    p= 3 
    b= [1,3,5]
    e= 1 # causes each subdomain to overlap the previous by 1
    K= 3
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [151]:
#subdomains contained in others
if iparam == 2:
    n= 10
    p= 5
    b= [1,1,5,5,7]
    e= [2,4,6,8,10] 
    K= 6
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [152]:
#overlap does not connect all subdomains
if iparam == 3:
    n= 9
    p= 4
    b= [1,3,6,8]
    e= [2,5,7,9]
    K= 3
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [153]:
if iparam == 4:
    n= 12
    p= 4 
    b= [1,2,5,9]
    e= [3,7,10,12]
    K= 3
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [154]:
print(p,b,e)
nn,nb,ne,index= fix_domains(n,p,b,e,iprint)

4 [1, 2, 4, 7] [3, 4, 8, 10]


#### Establish and solve forward system 

In [155]:
sread= True   #Read system of equations from iparam[iparam].csv. The solver will now ignore all of the parameters set above.
swrite= False  #Write system to iparam[iparam].csv

M,x,y,xstore= Linear_system(n,sread,iparam)
wpert= Make_wpert(n,p,K,sread,iparam) #Read or create wpert (depending on sread)
if iprint >= 4:
    print(M)
    print('y')
    print(y)        
    print('x')
    print(x)

if swrite:
    save_to_csv(n,p,K,0,nb,nn,ne,M,y,mpert,wpert,iparam)

['4']
[[1.04170220e+01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01
  3.96767474e-01 5.38816734e-01]
 [4.19194514e-01 1.06852195e+01 2.04452250e-01 8.78117436e-01
  2.73875932e-02 6.70467510e-01 4.17304802e-01 5.58689828e-01
  1.40386939e-01 1.98101489e-01]
 [8.00744569e-01 9.68261576e-01 1.03134242e+01 6.92322616e-01
  8.76389152e-01 8.94606664e-01 8.50442114e-02 3.90547832e-02
  1.69830420e-01 8.78142503e-01]
 [9.83468338e-02 4.21107625e-01 9.57889530e-01 1.05331653e+01
  6.91877114e-01 3.15515631e-01 6.86500928e-01 8.34625672e-01
  1.82882773e-02 7.50144315e-01]
 [9.88861089e-01 7.48165654e-01 2.80443992e-01 7.89279328e-01
  1.01032260e+01 4.47893526e-01 9.08595503e-01 2.93614148e-01
  2.87775339e-01 1.30028572e-01]
 [1.93669579e-02 6.78835533e-01 2.11628116e-01 2.65546659e-01
  4.91573159e-01 1.00533625e+01 5.74117605e-01 1.46728575e-01
  5.89305537e-01 6.99758360e-01]
 [1.02334429e-01 4.14055988e-01 6.94400158e-01 4.141

### Iterative strategy
$u^{\{k\}}= u^{\{k-1\}} + Br = u^{\{k-1\}} + B(f-Au^{\{k-1\}})$, $\quad k=1, \dots, K,$ where $B$ is an approximation to $A^{-1}$

<ins>Multiplicative<ins>
$A_i= R_i A R_i^\top \in \mathbb{R}^{m_i \times m_i}, \\
B_i= R_i^\top A_i^{-1} R_i  = R_i^\top ( R_i A R_i^\top)^{-1} R_i  \in \mathbb{R}^{n \times n}, \\
C_i= B_i A   \in \mathbb{R}^{n \times n},  \\
D_i= (I-C_i) \in \mathbb{R}^{n \times n},  \\
f_i= B_i f   \in \mathbb{R}^{n}, \; i=1,\dots,p.
$

In [156]:
#Construct domains, R,A,B,C and f
R= Rmatrices(n,p,nn,index,iprint)
A= Amatrices(M,R,n,p,nn,iprint)
B= Bmatrices(A,R,n,p,nn,iprint)
C= Cmatrices(M,B,n,p,iprint) 
D= Dmatrices_multiplicative(C,n,p,iprint)
f= fvector(B,y,n,p,iprint)

Print DD matrix 
[[[ 0.00000000e+00 -1.08313694e-17 -3.55143504e-19 -2.34797600e-02
   -1.40590027e-02 -4.64373355e-03 -1.52256494e-02 -2.96332017e-02
   -3.72974506e-02 -5.06848440e-02]
  [-5.35150477e-18  0.00000000e+00  4.56869876e-19 -8.01538577e-02
   -4.07261358e-04 -6.10217977e-02 -3.83908799e-02 -5.11871986e-02
   -1.14360700e-02 -1.50244391e-02]
  [-1.13433185e-17 -1.54062210e-18  0.00000000e+00 -5.77801722e-02
   -8.38457850e-02 -8.06524722e-02 -3.45956308e-03  3.31959968e-03
   -1.24974576e-02 -7.97998113e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00

#### Perform $K$  iterations computing subdomain contributions independently
$
u^{\{k+i/p\}} = u^{\{k+(i-1)/p\}} + B_i(f-Au^{\{k+(i-1)/p\}}) =D_i u^{\{k+(i-1)/p\}} + f_i \quad i=1,\dots,p$

In [157]:
#Perform D iteration
vstore= Diteration_multiplicative(D,f,n,p,K,iprint)
if iprint >= 4:
    print('vstore')
    print(vstore)


Outer iteration k, Inner iteration i 0 0
[[0.02812641 0.0467045  0.07933329 0.         0.         0.
  0.         0.         0.         0.        ]]
Outer iteration k, Inner iteration i 0 1
[[0.02812641 0.04469558 0.07785787 0.02478863 0.         0.
  0.         0.         0.         0.        ]]
Outer iteration k, Inner iteration i 0 2
[[ 0.02812641  0.04469558  0.07785787  0.01248406  0.07717579  0.05273767
  -0.01305289  0.08211002  0.          0.        ]]
Outer iteration k, Inner iteration i 0 3
[[ 0.02812641  0.04469558  0.07785787  0.01248406  0.07717579  0.05273767
  -0.02032271  0.07451813  0.03702722  0.07920325]]
Outer iteration  0
[[ 0.02812641  0.04469558  0.07785787  0.01248406  0.07717579  0.05273767
  -0.02032271  0.07451813  0.03702722  0.07920325]]
Outer iteration k, Inner iteration i 1 0
[[ 0.01920916  0.03780667  0.0614222   0.01248406  0.07717579  0.05273767
  -0.02032271  0.07451813  0.03702722  0.07920325]]
Outer iteration k, Inner iteration i 1 1
[[ 0.01920916  

#### Perform $K$ iterations using iteration matrices $S$ and $T$,  and  which compute the solution in all $p$ domains simultanously
Let $p=5$ and construct S and T, g and v such that

$S = \pmatrix{
 I   &  0   &   0   &  0   &   0   \cr
-D_2 &  I   &   0   &  0   &   0   \cr
 0   & -D_3 &   I   &  0   &   0   \cr
 0   &   0  &  -D_4 &  I   &   0   \cr
 0   &   0  &   0   & -D_5 &   I
} \in \mathbb{R}^{np \times np}$,
$T =
\pmatrix{
 0 & 0 & 0 & 0 & -D_1  \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0    \cr
 0 & 0 & 0 & 0 &  0
} \in \mathbb{R}^{np \times np}$, 
$g =
\pmatrix{
f_1  \cr
f_2  \cr
f_3  \cr
f_4  \cr
f_5
}
\in \mathbb{R}^{np}$,
$v^{\{k\}} =
\pmatrix{
u^{\{(k-1)+1/5\}}  \cr
u^{\{(k-1)+2/5\}}  \cr
u^{\{(k-1)+3/5\}}  \cr
u^{\{(k-1)+4/5\}}  \cr
u^{\{k\}}
}
\in \mathbb{R}^{np}$,

In [158]:
# Construct S, T and g and solve
S= Smatrix_multiplicative(D,n,p,iprint)
T= Tmatrix_multiplicative(D,n,p,iprint)
g= gvector_multiplicative(f,n,p,iprint)

wstore= STiteration(S,T,g,n,p,K,iprint)
if iprint >= 4:
    print('wstore')
    print(wstore)

Print S matrix
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
Print T matrix
[[ 0.          0.          0.         ...  0.0296332   0.03729745
   0.05068484]
 [ 0.          0.          0.         ...  0.0511872   0.01143607
   0.01502444]
 [ 0.          0.          0.         ... -0.0033196   0.01249746
   0.07979981]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
Print g vector
[[ 0.02812641]
 [ 0.0467045 ]
 [ 0.07933329]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.04575664]
 [ 0.07993995]
 [ 0.02481948]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 

#### Perform $K$ iterations as single matrix solve

$K=6$ iterations of multiplicative Schwarz can be written as the $npK$-dimensional system of equations $Uz=h$, where

$U=
\pmatrix{
 S & 0 & 0 & 0 & 0 & 0   \cr
 T & S & 0 & 0 & 0 & 0   \cr
 0 & T & S & 0 & 0 & 0   \cr
 0 & 0 & T & S & 0 & 0   \cr
 0 & 0 & 0 & T & S & 0   \cr
 0 & 0 & 0 & 0 & T & S
}
$
,
$z=
\pmatrix{
v^{\{1\}}  \cr
v^{\{2\}}  \cr
v^{\{3\}}  \cr
v^{\{4\}}  \cr
v^{\{5\}}  \cr
v^{\{6\}}
}
$
,
$
h=
\pmatrix{
g  \cr
g  \cr
g  \cr
g  \cr
g  \cr
g
}
$
.

In [159]:
# Construct U and h and solve
U= Umatrix_multiplicative(S,T,n,p,K,iprint) 
h= hvector_multiplicative(g,n,p,K,iprint)

# Solve using U and h
zstore= Uhsolve_multiplicative(U,h,n,p,K,iprint);
if iprint >= 4:
    print('zstore')
    print(zstore)

h vector
[[ 0.02812641]
 [ 0.0467045 ]
 [ 0.07933329]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.04575664]
 [ 0.07993995]
 [ 0.02481948]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.02043265]
 [ 0.08395934]
 [ 0.0565028 ]
 [-0.00663193]
 [ 0.08469071]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.01206386]
 [ 0.08072284]
 [ 0.05269031]
 [ 0.09058564]
 [ 0.02812641]
 [ 0.0467045 ]
 [ 0.07933329]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.04575664]
 [ 0.07993995]
 [ 0.02481948]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.02043265]
 [ 0.08395934]
 [ 0.0565028 ]
 

#### Compare Solutions from all three methods

In [160]:
solution_compare(xstore,vstore,wstore,zstore,iprint)

norm(exact - iterative) solutions    =  3.076840e-05
Difference between vstore and wstore =  6.778752e-17
Difference between wstore and zstore =  2.687422e-17
Difference between vstore and zstore =  7.091180e-17


### Add discretization error and compute error estimates and effectivity ratios

#### $K$  iterations computing subdomain contributions independently

In [161]:
# Set adjoint RHS
psi= np.ones((n,1))

# Perform D iteration with error: add random error to each subdomain solve
v_store,va_store,r_store= Diteration_approx(D,f,n,p,K,iprint,mpert,wpert)
if iprint >= 6:
    print(v_store)
    print(va_store)
    print(rstore)

Outer iteration  0
Outer iteration   0, Inner iteration   0
[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.22812641]
 [0.1267045 ]
 [0.13933329]
 [0.        ]
 [0.06      ]
 [0.02      ]
 [0.06      ]
 [0.06      ]
 [0.02      ]
 [0.06      ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [

 [ 0.        ]]
Outer iteration   2, Inner iteration   1
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.22812641]
 [ 0.1267045 ]
 [ 0.13933329]
 [ 0.        ]
 [ 0.06      ]
 [ 0.02      ]
 [ 0.06      ]
 [ 0.06      ]
 [ 0.02      ]
 [ 0.06      ]
 [ 0.43646685]
 [ 0.03881882]
 [ 0.06779347]
 [ 0.0103172 ]
 [ 0.07977722]
 [ 0.02038734]
 [ 0.06204669]
 [ 0.07806804]
 [ 0.03766612]
 [ 0.06229492]
 [ 0.44299975]
 [ 0.04935998]
 [ 0.08551232

#### Calculate the discretization error

In [162]:
# Perform adjoint solve 
phi_store= Diteration_adjoint(D,psi,n,p,K,iprint)

# Compute error estimate as (r,phi)
qoi_discretization_error_estimate= r_store.T @ phi_store

# Compute error and error in QoI directly
vesoln=  v_store[n*p*K + n*(p-1):n*p*(K+1),0:1]
vsoln=  va_store[n*p*K + n*(p-1):n*p*(K+1),0:1]
verror= vesoln - vsoln

qoi_discretization_error= psi.T @ verror

k, i, pindex, id1, id2: 3 4 0 110 120
Diteration_adjoint: special case 

k, i, pindex, id1, id2: 3 3 3 100 110
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+

 [ 1.00000000e+00]]
k, i, pindex, id1, id2: 2 3 3 60 70
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000

 [ 1.00000000e+00]]
k, i, pindex, id1, id2: 1 3 3 20 30
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-2.05816728e-04]
 [-4.64477194e-04]
 [-3.32049175e-04]
 [-4.25681596e-04]
 [-1.70783184e-03]
 [-3.85863025e-03]
 [-2.41309443e-20]
 [-2.46563195e-19]
 [-4.02215464e-20]
 [-1.29462713e-20]
 [ 2.58881176e-35]
 [-1.60617562e-19]
 [-5.26639633e-21]
 [-3.48179597e-04]
 [-1.56621424e-03]
 [-3.50623733e-03]
 [-1.36564519e-03]
 [-1.86575433e-03]
 [ 4.46723452e-03]
 [ 3.14234737e-03]
 [ 1.48289248e-02]
 [ 3.51431683e-18]
 [-3.94020506e-18]
 [-4.72821504e-20]
 [-1.35773434e-03]
 [-3.43737575e-03]
 [-1.13986518e-03]
 [-1.42632581e-03]
 [ 5.02031561e-03]
 [ 3.89394911

#### Calculate the effectivity ratio

In [163]:
effectivity_ratio_disc = qoi_discretization_error_estimate / qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}')
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

QoI_discretization_error          = -5.734394e-02
QoI_discretization_error_estimate = -5.734394e-02
Effectivity ratio                 =         1.000


#### Calculate the total error

In [164]:
total_error= xstore - vsoln
qoi_total_error= psi.T @ total_error

phi= np.linalg.solve(M.T,psi)
r= y - M @ vsoln
qoi_total_error_estimate= r.T @ phi

#### Calculate the effectivity ratio

In [165]:
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

QoI_total_error                   = -5.730823e-02
QoI_total_error_estimate          = -5.730823e-02
Effectivity ratio                 =         1.000


#### Calculate the iteration error

In [166]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

QoI_iteration_error               =  3.571300e-05


#### $K$ iterations as single matrix solve

In [167]:
# Perform U iteration with error: add random error to linear system solution
zexact,zapprox,zresid= Uhsolve_multiplicative_approx(U,h,n,p,K,iprint,mpert,wpert)

# Adjoint data (Adjoint system is of size npK)
psi_npK= np.zeros((n*p*K,1))
psi_npK[n*p*K - n:n*p*K,0]= psi.reshape(n)

#  Solve adjoint equation
phi_npK= np.linalg.solve(U.T,psi_npK)
if iprint >= 2:
    print(f'Norm of difference between phi_npK and phi_store = {np.linalg.norm(phi_npK-phi_store):13.6e}')

# Compute error estimate
qoi_discretization_error_estimate= phi_npK.T @ zresid

# Compute error and error in QoI directly
xsoln= zexact[n*p*K - n:n*p*K,0:1]
zsoln= zapprox[n*p*K - n:n*p*K,0:1]
zerror= xsoln-zsoln
qoi_discretization_error= psi.T @ zerror

[ 0.02812641  0.0467045   0.07933329  0.          0.          0.
  0.          0.          0.          0.          0.02812641  0.04469558
  0.07785787  0.02478863  0.          0.          0.          0.
  0.          0.          0.02812641  0.04469558  0.07785787  0.01248406
  0.07717579  0.05273767 -0.01305289  0.08211002  0.          0.
  0.02812641  0.04469558  0.07785787  0.01248406  0.07717579  0.05273767
 -0.02032271  0.07451813  0.03702722  0.07920325]
[ 0.01920916  0.03780667  0.0614222   0.01248406  0.07717579  0.05273767
 -0.02032271  0.07451813  0.03702722  0.07920325  0.01920916  0.03803363
  0.06158888  0.00968361  0.07717579  0.05273767 -0.02032271  0.07451813
  0.03702722  0.07920325  0.01920916  0.03803363  0.06158888  0.0096028
  0.07810922  0.04630103 -0.0185469   0.07573675  0.03702722  0.07920325
  0.01920916  0.03803363  0.06158888  0.0096028   0.07810922  0.04630103
 -0.01884989  0.07546822  0.03968189  0.08067222]
[ 0.01906954  0.03827241  0.0618772   0.0096028  

#### Calculate the effectivity ratio

In [168]:
effectivity_ratio_disc= qoi_discretization_error_estimate/qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error[0,0]:13.6e}' )
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc[0,0]:13.3f}')

QoI_discretization_error          = -2.054190e-02
QoI_discretization_error_estimate = -2.054190e-02
Effectivity ratio                 =         1.000


#### Calculate the total error

In [169]:
total_error= xstore - zsoln
qoi_total_error= psi.T @ total_error

r= y - M@zsoln
qoi_total_error_estimate= r.T @ phi

#### Calculate the effectivity ratio

In [170]:
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

QoI_total_error                   = -2.050619e-02
QoI_total_error_estimate          = -2.050619e-02
Effectivity ratio                 =         1.000


#### Calculate iteration error

In [171]:
qoi_iteration_error= qoi_total_error - qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

QoI_iteration_error               =  3.571300e-05
